In [1]:
%load_ext pycodestyle_magic
%load_ext mypy_ipython
%pycodestyle_on

In [2]:
import doctest

In [3]:
import os
import struct
import itertools
from operator import itemgetter
from tempfile import NamedTemporaryFile


class StructField:

    def __init__(self, format, offset):
        self.format = format
        self.offset = offset

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            r = struct.unpack_from(self.format, instance._buffer, self.offset)
            return r[0] if len(r) == 1 else r


class StructureMeta(type):

    def __init__(self, clsname, bases, clsdict):
        fields = getattr(self, '_fields', [])
        byte_order = ''
        offset = 0
        for format, fieldname in fields:
            if isinstance(format, StructureMeta):
                setattr(self, fieldname, NestedStruct(fieldname, format, offset))  # noqa: E501
                offset += format.struct_size
            else:
                if format.startswith(('<', '>', '!', '@')):
                    byte_order = format[0]
                    format = format[1:]

                format = byte_order + format
                setattr(self, fieldname, StructField(format, offset))
                offset += struct.calcsize(format)

        setattr(self, 'struct_size', offset)


class Structure(metaclass=StructureMeta):

    def __init__(self, bytedata):
        self._buffer = memoryview(bytedata)

    @classmethod
    def from_file(cls, f):
        return cls(f.read(cls.struct_size))


class NestedStruct:

    def __init__(self, name, struct_type, offset):
        self.name = name
        self.struct_type = struct_type
        self.offset = offset

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            data = instance._buffer[self.offset:self.offset + self.struct_type.struct_size]  # noqa: E501
            result = self.struct_type(data)
            setattr(instance, self.name, result)
            return result


class SizedRecord:

    def __init__(self, bytedata):
        self._buffer = memoryview(bytedata)

    @classmethod
    def from_file(cls, f, size_fmt, includes_size=True):
        sz_nbytes = struct.calcsize(size_fmt)
        sz_bytes = f.read(sz_nbytes)
        sz, *_ = struct.unpack(size_fmt, sz_bytes)
        buf = f.read(sz - includes_size * sz_nbytes)
        return cls(buf)

    def iter_as(self, code):
        if isinstance(code, str):
            s = struct.Struct(code)
            for off in range(0, len(self._buffer), s.size):
                yield s.unpack_from(self._buffer, off)
        elif isinstance(code, StructureMeta):
            size = code.struct_size
            for off in range(0, len(self._buffer), size):
                data = self._buffer[off:off+size]
                yield code(data)


class Point(Structure):
    _fields = [
        ('<d', 'x'),
        ('d', 'y')
    ]


class PolyHeader(Structure):
    _fields = [
        ('<i', 'file_code'),
        (Point, 'min'),
        (Point, 'max'),
        ('i', 'num_polys')
    ]


def write_polys(filename, polys):
    flattened = list(itertools.chain(*polys))
    xs, ys = sorted(x for x, _ in flattened), sorted(y for _, y in flattened)
    min_x, max_x = xs[0], xs[-1]
    min_y, max_y = ys[0], ys[-1]
    with open(filename, 'wb') as f:
        f.write(struct.pack(
            '<iddddi', 0x1234, min_x, min_y, max_x, max_y, len(polys)))
        for poly in polys:
            size = len(poly) * struct.calcsize('<dd')
            f.write(struct.pack('<i', size + 4))
            for pt in poly:
                f.write(struct.pack('<dd', *pt))


def read_polys(filename):
    polys = []
    with open(filename, 'rb') as f:
        phead = PolyHeader.from_file(f)
        for n in range(phead.num_polys):
            rec = SizedRecord.from_file(f, '<i')
            poly = [(p.x, p.y) for p in rec.iter_as(Point)]
            polys.append(poly)

    return polys


polys = [
    [(1.0, 2.5), (3.5, 4.0), (2.5, 1.5)],
    [(7.0, 1.2), (5.1, 3.0), (0.5, 7.5), (0.8, 9.0)],
    [(3.4, 6.3), (1.2, 0.5), (4.6, 9.2)]
]

write_polys('polys.bin', polys)


"""

>>> f = open('polys.bin', 'rb')
>>> phead = PolyHeader.from_file(f)
>>> print(f'0x{phead.file_code:0x}')
0x1234
>>> phead.min.x
0.5
>>> phead.min.y
0.5
>>> phead.max.x
7.0
>>> phead.max.y
9.2
>>> phead.num_polys
3
>>> poly_data = [SizedRecord.from_file(f, '<i') for _ in range(phead.num_polys)]
>>> poly_data  # doctest: +ELLIPSIS
[<__main__.SizedRecord object at ...>, ...]
>>> for n, poly in enumerate(poly_data):  # doctest: +NORMALIZE_WHITESPACE
...     print(f'polygon {n}')
...     for p in poly.iter_as('<dd'):
...         print('\\t', p)
polygon 0
    (1.0, 2.5)
    (3.5, 4.0)
    (2.5, 1.5)
polygon 1
    (7.0, 1.2)
    (5.1, 3.0)
    (0.5, 7.5)
    (0.8, 9.0)
polygon 2
    (3.4, 6.3)
    (1.2, 0.5)
    (4.6, 9.2)
>>> polys = read_polys('polys.bin')
>>> for p in polys:
...     print(p)
[(1.0, 2.5), (3.5, 4.0), (2.5, 1.5)]
[(7.0, 1.2), (5.1, 3.0), (0.5, 7.5), (0.8, 9.0)]
[(3.4, 6.3), (1.2, 0.5), (4.6, 9.2)]
>>> f.close()
"""  # noqa: E501

result = doctest.testmod()
os.remove('polys.bin')
result

TestResults(failed=0, attempted=14)